<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/A2_adapter_seq_bn_classification_C_5_percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This tests using an adapter for pretraining





# Setup

In [2]:
# Pretraining Dataset Settings
# pretrain_dataset_name = "BigTMiami/amazon_split_25M_reviews_20_percent_condensed"
# # 5% of 20% creates a 1% - I think this should take 15 minutes
# pretrain_percentage = "5"
# pretrain_train_split = f"train[:{pretrain_percentage}%]"
# pretrain_eval_split = "dev"

# # Adapter settings
experiment_tag = "A2"
adapter_short_desc = "classification"
adapter_type = "seq_bn" # could be "lora", etc.

# adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}"
# adapter_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_P_{pretrain_percentage}"
# print(f"Adapter Hub Name: {adapter_hub_name}")

# Classification Dataset Settings
classification_dataset_name = "BigTMiami/amazon_helpfulness"
classification_train_percentage = "5"
classification_train_split = f"train[:{classification_train_percentage}%]"
classification_eval_split = "dev"

classification_adapter_hub_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_C_{classification_train_percentage}"
classification_adapter_name = f"{experiment_tag}_adapter_{adapter_type}_{adapter_short_desc}_C_{classification_train_percentage}"

print(f"Adapter Hub Name: {classification_adapter_hub_name}")

# Adapter to load from hub
# adapter_model_hub_name = "BigTMiami/adapter_classification_no_pretraining_20_percent"

# Classification Model Hub Name if pushed \ Output Dir
# classification_model_hub_name = "classification_model_hub_name"

Adapter Hub Name: A2_adapter_seq_bn_classification_C_5


In [3]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [4]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [5]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00


In [6]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 233 MB


In [7]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [9]:
from datasets import load_dataset

classification_train_dataset, classification_eval_dataset= load_dataset(classification_dataset_name, split=[classification_train_split,classification_eval_split])

print_gpu_utilization()
print(classification_train_dataset)
print(classification_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5763
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [11]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_no_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'heads.default.3.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# adapter_hub_name = "adapter_classifier_from_prtrained_test"
# adapter_name = model.load_adapter("BigTMiami/domain_adapter_test")
# # Add head for masked language modeling
# model.add_classification_head(
#     adapter_name,
#     num_labels=2,
#     id2label={ 0: "unhelpful", 1: "helpful"}, overwrite_ok=True
#   )

# # Set the adapter to be used for training
# model.train_adapter(adapter_name)

In [12]:
# Add a new adapter
classficiation_no_pretrain_model.add_adapter(classification_adapter_name, config=adapter_type)

# Add head for classification modeling
classficiation_no_pretrain_model.add_classification_head(
    classification_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_no_pretrain_model.train_adapter(classification_adapter_name)

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 233 MB


In [14]:
from transformers import DataCollatorWithPadding

classification_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Train

In [16]:
from transformers import TrainingArguments

classification_training_args = TrainingArguments(
    output_dir=classification_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=250,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [17]:
from adapters import AdapterTrainer

classification_trainer = AdapterTrainer(
    model=classficiation_no_pretrain_model,
    args=classification_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [19]:
classification_training_results = classification_trainer.train()
print(classification_training_results)

Step,Training Loss
250,0.351000
500,0.323200
750,0.324600
1000,0.316100


TrainOutput(global_step=1083, training_loss=0.32670273785225684, metrics={'train_runtime': 354.241, 'train_samples_per_second': 48.806, 'train_steps_per_second': 3.057, 'total_flos': 4586842797442206.0, 'train_loss': 0.32670273785225684, 'epoch': 3.0})


In [20]:
classification_eval_results = classification_trainer.evaluate()
classification_eval_loss = classification_eval_results["eval_loss"]
classification_eval_f1 = classification_eval_results["eval_f1_macro"]
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.4f}")
print(classification_eval_results)

Classification no Pretraining Eval F1:59.55
Classification no Pretraining Eval Loss: 0.35
{'eval_loss': 0.35461702942848206, 'eval_accuracy': 0.8634, 'eval_f1_macro': 0.5955343677941605, 'eval_runtime': 47.9376, 'eval_samples_per_second': 104.302, 'eval_steps_per_second': 6.529, 'epoch': 3.0}


In [21]:
classficiation_no_pretrain_model.push_adapter_to_hub(
    classification_adapter_hub_name,
    classification_adapter_name,
    datasets_tag=classification_dataset_name
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/A2_adapter_seq_bn_classification_C_5/commit/427470a155c06e4adbeb0171fc2f57c8c22116f7', commit_message='Upload model', commit_description='', oid='427470a155c06e4adbeb0171fc2f57c8c22116f7', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
import gc

print_gpu_utilization()

del classficiation_no_pretrain_model
del classification_trainer
gc.collect()

print_gpu_utilization()

GPU 9629 MB
GPU 9629 MB


In [23]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
